In [62]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import os

In [63]:
model_name = "huawei-noah/TinyBERT_General_4L_312D"

In [64]:
model = SentenceTransformer(model_name)

In [65]:
dat = model.encode(["hello i love this", "Yeh this is cool"])

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [66]:
df = pd.read_csv("data.csv")
df.head()

,content,class
0,Project discussion scheduled for Thursday at 1...,imp_no_urgent
1,Weekly staff meeting on Monday at 9 AM confirmed.,imp_no_urgent
2,Doctor's appointment on Friday at 3 PM reminde...,imp_no_urgent
3,Quarterly review set for next Wednesday at 2 PM.,imp_no_urgent
4,Conference call with marketing team on Tuesday...,imp_no_urgent


In [67]:
enc = OneHotEncoder()

In [68]:
encoded_df = pd.DataFrame(enc.fit_transform(df[["class"]]).toarray(), columns=enc.get_feature_names_out(["class"]))
df = pd.concat([df, encoded_df], axis=1)
df.drop(columns=["class"], inplace=True)

In [69]:
df.head()

,content,class_imp_no_urgent,class_low,class_moderate,class_urgent
0,Project discussion scheduled for Thursday at 1...,1.0,0.0,0.0,0.0
1,Weekly staff meeting on Monday at 9 AM confirmed.,1.0,0.0,0.0,0.0
2,Doctor's appointment on Friday at 3 PM reminde...,1.0,0.0,0.0,0.0
3,Quarterly review set for next Wednesday at 2 PM.,1.0,0.0,0.0,0.0
4,Conference call with marketing team on Tuesday...,1.0,0.0,0.0,0.0


In [70]:
X_sen = df['content']
Y = df.iloc[:, 1:]

In [71]:
Y.head()

,class_imp_no_urgent,class_low,class_moderate,class_urgent
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0


In [72]:
Y.shape

(4300, 4)

In [73]:
X = model.encode(X_sen)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [74]:
X.shape

(4300, 312)

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.15, random_state=42)

In [77]:
class ClassifierHead(nn.Module):
    def __init__(self, in_features, p = .2):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=150),
            nn.ReLU(),
            nn.Dropout(p=p),
            nn.Linear(in_features=150, out_features=80),
            nn.Dropout(p=.4),
            nn.ReLU(),
            nn.Linear(in_features=80, out_features=4)
        )

    def forward(self, x):
        y = self.model(x)
        return y


In [78]:
class CustomDataset(Dataset):
  def __init__(self, X, Y):
    super().__init__()
    self.X = X
    self.y = Y

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    # Return the index of the one-hot encoded label
    return torch.tensor(self.X[idx], dtype=torch.float32), torch.argmax(torch.tensor(self.y.iloc[idx].values, dtype=torch.float32))

In [80]:
dataloader = DataLoader(CustomDataset(X_train, y_train), batch_size=16, shuffle=True, )

In [81]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [82]:
def evaluate(model, Y_test, X_test):
    model.eval()
    with torch.no_grad():
        predictions = model(torch.tensor(X_test, dtype=torch.float32))
        Y_pred  = predictions.numpy()

        Y_test_labels = np.argmax(Y_test.values, axis=1)

        accuracy = accuracy_score(Y_test_labels, Y_pred.argmax(axis=1))
        f1 = f1_score(Y_test_labels, predictions.argmax(axis=1), average='weighted')

        print(f'Accuracy: {accuracy:.2f}')
        print(f'F1 Score: {f1:.2f}')

        return f1

In [83]:
def train(model, criterion, epochs, optimizer, dataloader, device, Y_test, X_test):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0

        best_f1 = 1

        for x, y in dataloader:
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            prediction = model(x)
            loss = criterion(prediction, y)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        epoch_loss /= len(dataloader)
        if (epoch + 1) % 10 == 0:
            print('-------------------------------------------------------')
            print(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}")
            f1 = evaluate(model, y_test, X_test)

            # CheckPoint
            if not os.path.isdir('Models'):
              os.mkdir('Models')

            if f1 < best_f1:
                torch.save(model.state_dict(), 'Models/best_model.pth')


In [84]:
model = ClassifierHead(312)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
epochs = 100
train(model, criterion, epochs, optimizer, dataloader, device, y_test, X_test)

-------------------------------------------------------
Epoch [10/100], Loss: 0.2945
Accuracy: 0.84
F1 Score: 0.84
-------------------------------------------------------
Epoch [20/100], Loss: 0.1400
Accuracy: 0.84
F1 Score: 0.84
-------------------------------------------------------
Epoch [30/100], Loss: 0.0850
Accuracy: 0.85
F1 Score: 0.85
-------------------------------------------------------
Epoch [40/100], Loss: 0.0563
Accuracy: 0.85
F1 Score: 0.85
-------------------------------------------------------
Epoch [50/100], Loss: 0.0419
Accuracy: 0.85
F1 Score: 0.85
-------------------------------------------------------
Epoch [60/100], Loss: 0.0461
Accuracy: 0.85
F1 Score: 0.85
-------------------------------------------------------
Epoch [70/100], Loss: 0.0430
Accuracy: 0.85
F1 Score: 0.85
-------------------------------------------------------
Epoch [80/100], Loss: 0.0360
Accuracy: 0.86
F1 Score: 0.86
-------------------------------------------------------
Epoch [90/100], Loss: 0.